In [25]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
from ipywidgets import *
from bqplot import *
from bqplot.interacts import *

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

In [26]:
#INPUTS    ********************
asset_list=['IBE SM Equity', 'SAN SM Equity', 'ITX SM Equity', 'TEF SM Equity']
cur='EUR'
start_date = datetime.datetime(2018, 5, 12)
end_date = datetime.datetime(2018, 6, 12)

In [27]:
def norm_corr(as1, as2, st_date, e_date, cur):
    price = d.px_last(start = st_date, end = e_date, fill='prev', currency = cur)
    price = f.zscore(price)
    base_price = f.value(price, u.list(as1))
    correlation = f.corr(base_price, price)

    request = bql.Request(as2, {'Cor': correlation})
    res = bq.execute(request)
    cor_res = res[0].df()
    
    return cor_res.iloc[0]['Cor']

In [28]:
###############    MAIN    ###############

#Calculate Correlation Matrix
count = 0
for asset1 in asset_list:
    aux_cor_list=[]
    for asset2 in asset_list:
        aux_cor_list.append(norm_corr(asset1,asset2,start_date,end_date,cur))
        
    if count == 0:
        Correl_Matrix = pd.DataFrame({asset1: pd.Series(aux_cor_list, index = asset_list)})
    else:
        aux_Correl_Matrix = pd.DataFrame({asset1: pd.Series(aux_cor_list, index = asset_list)})
        Correl_Matrix = pd.concat([Correl_Matrix, aux_Correl_Matrix],axis = 1)
        
    count += 1

#Display Heatmap    
names = pd.Index(asset_list)
s = pd.Series(np.sqrt(np.diag(Correl_Matrix)), index=Correl_Matrix.index)

tkx = np.kron(np.ones_like(Correl_Matrix.index), Correl_Matrix.index)
tky = np.kron(Correl_Matrix.index, np.ones_like(Correl_Matrix.index))

sc_cov_x = OrdinalScale()
sc_cov_y = OrdinalScale(reverse=True)
col_sc = ColorScale(min=-1.0, max=1.0, colors=['#998ec3', '#f7f7f7', '#f1a340'])
ax_cov_x = Axis(scale=sc_cov_x, side='top')
ax_cov_y = Axis(scale=sc_cov_y, orientation='vertical')
grid_map = GridHeatMap(color=Correl_Matrix, row=names, column=names,
                       scales={'column': sc_cov_x, 'row': sc_cov_y, 'color': col_sc})

lab = Label(x=np.kron(np.ones_like(names), names), y=np.kron(names, np.ones_like(names)),
            text=['{:0.2f}'.format(Correl_Matrix.loc[tkx[i],tky[i]]).replace('-0', '-').lstrip('0') if tkx[i] != tky[i] else '{:0.1%}'.format(s.loc[tkx[i]]) for i in range(len(tkx))],
            scales={'x': sc_cov_x, 'y': sc_cov_y}, align='middle',
            colors=['black' if tkx[i] != tky[i] else 'yellow' for i in range(len(tkx))])

grid_fig = Figure(marks=[grid_map, lab], axes=[ax_cov_x, ax_cov_y], padding_y=0.0)
grid_fig.layout.width = '800px'
grid_fig.layout.height = '800px'

display(grid_fig)

#col_sc = ColorScale()
#grid_map = GridHeatMap(color=Correl_Matrix, scales={'color': col_sc})

#Figure(marks=[grid_map], padding_y=0.0)